<a href="https://colab.research.google.com/github/NgaAdrain/TEAM_Enter_Text_Here/blob/master/Data_Varification/20200707_Test_Model_4_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

csv -> data<br>



In [3]:
# Import the TensorFlow
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
tf.__version__


'2.2.0'

윈도우의 크기를 32 -> 32개의 row를 하나의 window로<br>
RPM의 최대값을 5000으로 설정 -> for 정규화<br>
속도의 최대값을 210으로 설정 -> for 정규화

In [4]:
WINDOW_SIZE = 32
MAX_RPM = 5000
MAX_VELO = 210
#첫 번째 학습 데이터 -> 아마 좌회전 데이터
data_file = pd.read_csv('/content/drive/My Drive/20200709_124409.csv',encoding='utf-8')
#data_file = pd.read_csv('/content/20200707_161530.csv',encoding='utf-8')
data_file.drop(['Timestamp'],axis = 'columns',inplace=True)
#print(data_file)
#for d in data_file:
#  print(data_file)

In [5]:
label = data_file.pop('Label')
#list(data_file.values)

In [7]:
#두 번째 학습 데이터 -> 아마 우회전 데이터
train_file = pd.read_csv('/content/drive/My Drive/20200709_123806.csv',encoding='utf-8')
#train_file = pd.read_csv('/content/20200707_160439.csv',encoding='utf-8')
train_file.RPM = train_file.RPM/MAX_RPM
train_file.Velocity = train_file.Velocity/MAX_VELO
train_file.drop(['Timestamp'],axis = 'columns',inplace=True)
train_file_label = train_file.pop('Label')
#train_file_label = train_file.pop('Label1')

In [8]:
train_array = np.delete(train_file.values,-1,0)
train_label = np.delete(train_file_label.values,-1,0)

In [9]:
#테스트 데이터 -> 광역 이동
test_file = pd.read_csv('/content/drive/My Drive/20200709_121628.csv',encoding='utf-8')
#test_file = pd.read_csv('/content/20200707_164523.csv',encoding='utf-8')
#test_file.drop(['Label1'],axis = 'columns',inplace=True)
test_file.RPM = test_file.RPM/MAX_RPM
test_file.Velocity = test_file.Velocity/MAX_VELO
test_file_timestamp = test_file.pop('Timestamp')
#test_file_label = test_file.pop('Label1')
test_file_label = test_file.pop('Label')
test_array = np.delete(test_file.values,-1,0)
test_label = np.delete(test_file_label.values,-1,0)

두 번째 학습 데이터 -> 처리 과정(windowing)

In [10]:

xdataset = []
ydataset = []
#in range()
for i in range(WINDOW_SIZE, train_array.shape[0] + 1, 1):
  xdataset.append(train_array[i-WINDOW_SIZE:i])
  ydataset.append(train_label[i-1])
xtrain = np.array(xdataset) #32개의 데이터를 1개의 window 단위로 설정
ytrain = np.array(ydataset) #각 window의 마지막 원소를 대표 label로 지정
print(xtrain.shape)
print(ytrain.shape)

(9362, 32, 7)
(9362,)


테스트 데이터 -> 처리 과정(windowing)

In [11]:
xdataset = []
ydataset = []
#in range()
for i in range(WINDOW_SIZE, test_array.shape[0] + 1, 1):
  xdataset.append(test_array[i-WINDOW_SIZE:i])
  ydataset.append(0)
xtest = np.array(xdataset) #32개의 데이터를 1개의 window 단위로 설정
ytest = np.array(ydataset)
print(xtest.shape)
print(ytest.shape)

(37227, 32, 7)
(37227,)


모든 value가 0\~1사이(또는 -1~1 사이)의 값이 되도록 나누기(정규화 과정)

In [12]:
data_file.RPM = data_file.RPM/MAX_RPM
data_file.Velocity = data_file.Velocity/MAX_VELO

test_dataset의 sliding window를 정하기 ex) 32 -> 2차원을 3차원으로

In [13]:
print(type(data_file))
print(type(data_file.values))
np_array = np.delete(data_file.values,-1,0)
label = np.delete(label.values,-1,0)
print(type(np_array))

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [14]:
xdataset = []
ydataset = []
#in range()
for i in range(WINDOW_SIZE, np_array.shape[0] + 1, 1):
  xdataset.append(np_array[i-WINDOW_SIZE:i])
  ydataset.append(label[i-1])
np_xtest = np.array(xdataset) #32개의 데이터를 1개의 window 단위로 설정
np_ytest = np.array(ydataset) #각 window의 마지막 원소를 대표 label로 지정
print(np_xtest.shape)
print(np_ytest.shape)
#test_dataset = tf.data.Dataset.from_tensor_slices((data_file.values, label.values))
#print(type(data_file.values[0]))
#print(type(test_dataset))
#test_dataset = test_dataset.window(32,shift=1)
#data_file.values.shape
#for ds in test_dataset:
#  print(ds)

(10342, 32, 7)
(10342,)


가장 기본적인 모델<br>
데이터를 펼쳐서 128개의 tensor(뉴런)을 가지는 층을 추가<br>
3개의 출력값을 확인할 3개의 tensor(뉴런)을 가지는 층을 추가 -> softmax로
확률화 시킴(다 더해서 1)

In [15]:
#for val, lab in test_dataset.take(40):
#  print('Values: {}, Label: {}'.format(val,lab))
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(32, 7)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

첫 번째 데이터 학습

In [16]:
model.fit(np_xtest, np_ytest, epochs=5)

Epoch 1/5
324/324 [==============================] - 1s 2ms/step - loss: 0.0979 - accuracy: 0.9733
Epoch 2/5
324/324 [==============================] - 1s 2ms/step - loss: 0.0137 - accuracy: 0.9967
Epoch 3/5
324/324 [==============================] - 1s 2ms/step - loss: 0.0048 - accuracy: 0.9992
Epoch 4/5
324/324 [==============================] - 1s 2ms/step - loss: 0.0023 - accuracy: 0.9997
Epoch 5/5
324/324 [==============================] - 1s 2ms/step - loss: 0.0014 - accuracy: 0.9998


두 번재 데이터 학습

In [17]:
model.fit(xtrain, ytrain, epochs=5)

Epoch 1/5
293/293 [==============================] - 1s 2ms/step - loss: 0.0073 - accuracy: 0.9978
Epoch 2/5
293/293 [==============================] - 1s 2ms/step - loss: 0.0011 - accuracy: 0.9999
Epoch 3/5
293/293 [==============================] - 1s 2ms/step - loss: 3.0688e-04 - accuracy: 1.0000
Epoch 4/5
293/293 [==============================] - 1s 2ms/step - loss: 1.9717e-04 - accuracy: 1.0000
Epoch 5/5
293/293 [==============================] - 1s 2ms/step - loss: 1.3514e-04 - accuracy: 1.0000


학습 테스트 및 예측

In [22]:
test_loss, test_acc = model.evaluate(xtest,  ytest, verbose=2)
predictions = model.predict(xtest)
count = 0
print(type(test_file_timestamp[0]))
txt = 'Timestamp,Pred_Direction,Real_Direction\n'
for temp in xtest:
  #print(test_file_timestamp[count],end=' ')
  txt = txt + test_file_timestamp[count] + ' '
  if(np.argmax(predictions[count]) == 0):
  #  print('straight')
    txt = txt + ',straight'
  elif(np.argmax(predictions[count]) == 1):
  #  print('left')
    txt = txt + ',left'
  else:
    txt = txt + ',right'
  if(test_file_label[count] == 0):
    txt = txt + ',straight\n'
  elif(test_file_label[count] == 1):
    txt = txt + ',left\n'
  else:
    txt = txt + ',straight\n'
  #  print('right')
  count = count + 1

f = open("/content/drive/My Drive/0709_test_output.csv",'w')
f.write(txt)
f.close()

1164/1164 - 1s - loss: 1.0065 - accuracy: 0.8680
<class 'str'>


해야 하는것!<br>
데이터를 한 csv가 아닌 원하는 csv를 전부 돌아가게 해야함.<br>
읽기-> 처리 -> 학습 -> 다음파일 읽기 -> 처리 -> 학습 =>> 반복
를 구현<br>
그리고 지금은 단순히 Flatten -> 128 layer 통과 -> 결과<br>
정확도가 그렇게 높지 않다!
